# GOPH 619 Lab1


### Question 1:
Express the number 1.481 in floating point binary format, as
discussed in class. Clearly show the binary strings representing the
exponent and the mantissa. At what digit does the “true error” in the
mantissa reach zero? Do you think that your result has zero round-off
error? Explain

In [1]:
#### decimal to floating point



2.5

### Question 2

Show the Taylor series for sin(x) for zero- through fifth-order.
Evaluate sin(x) for xi+1 = π/3 using a base point of xi = π/4 using the zero-
through fifth-order Taylor series expansions. Calculate the true and
approximate relative error for each estimate. Why is π/4 a convenient
base point for this function?

### Question 3

Generate sample values of ex from x = 0 to x = 5 using a step
size Δx = 0.5. Estimate the value of e2.4 using zero- through fourth-order
polynomial interpolation. Do this using the Vandermonde matrix,
Newton IP and Lagrange IP methods. Calculate the true and approximate
relative error for each estimate